In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

C:\Users\agnes\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\agnes\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [5]:
sentences= ["Aspiring human resources", "seeking human resources"]

In [6]:
import pandas as pd 
data= pd.read_csv("C:\\Users\\agnes\\Documents\\apziva_ai_residency\\project3\\data\\potential-talents.csv")
words=data.job_title
text_array=words.array

In [8]:
for text in text_array:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(text, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [10]:
outputs = model(**tokens)
outputs.keys()


odict_keys(['last_hidden_state', 'pooler_output'])

In [11]:
embeddings = outputs.last_hidden_state

In [12]:
embeddings.shape

torch.Size([104, 128, 768])

In [13]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([104, 128])

In [14]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([104, 128, 768])

In [15]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [16]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([104, 128, 768])

In [17]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([104, 768])

In [18]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([104, 768])

In [19]:
mean_pooled = summed / summed_mask

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.4616598 , 0.5977877 , 0.3937672 , 0.35666168, 0.5832867 ,
        0.6913141 , 0.47467756, 0.6913141 , 0.3460192 , 0.339964  ,
        0.42487437, 0.40287083, 0.9999999 , 0.99999976, 0.4616598 ,
        0.5977877 , 0.3937672 , 0.99999976, 0.4616598 , 0.5977877 ,
        0.3937672 , 0.35666168, 0.5832867 , 0.6913141 , 0.47467756,
        0.68339944, 0.26227215, 0.68339944, 0.26227215, 0.99999976,
        0.4616598 , 0.5977877 , 0.3937672 , 0.35666168, 0.5832867 ,
        0.6913141 , 0.47467756, 0.6913141 , 0.3460192 , 0.339964  ,
        0.42487437, 0.40287083, 0.99999976, 0.4616598 , 0.5977877 ,
        0.3937672 , 0.35666168, 0.5832867 , 0.6913141 , 0.47467756,
        0.6913141 , 0.3460192 , 0.339964  , 0.42487437, 0.40287083,
        0.99999976, 0.5977877 , 0.3937672 , 0.5832867 , 0.47467756,
        0.3460192 , 0.339964  , 0.42487437, 0.40287083, 0.7339076 ,
        0.5122349 , 0.37131047, 0.34579676, 0.59219885, 0.41270012,
        0.78995043, 0.6977759 , 0.33822364, 0.48